In [1]:
###############################################################################
### pthon imports                                                           ###
###############################################################################

from os import path
from requests import get
from bs4 import BeautifulSoup
import os
import pandas as pd
import re

In [2]:
def find_file(
    file_name='path/file.txt',
    cache=False,
    cache_age=None
    ):

    if cache==False:
        return None
    
    return path.exists(file_name)

In [14]:
def get_soup(
    url='https://github.com/search?o=desc&p=1&q=advent+of+code&s=stars&type=Repositories',
    headers={'User-Agent': 'Nothing suspicious'},
    file_name='soupfiles/gitsearch.txt',
    cache=False,
    cache_age=None,
    soup_slurper='*'
):
    # if we already have the data, read it locally
    file_found = find_file(file_name=file_name, cache=cache, cache_age=cache_age)
    if file_found:
        with open(file_name) as f:
            return BeautifulSoup(f.read())

    # otherwise go fetch the data
    response = get(url, headers=headers)
    soup = BeautifulSoup(response.text)
    slurps = soup.select(soup_slurper)

    # save it for next time
    with open(file_name, 'w') as f:
        f.write(str(slurps[0]))
        if len(slurps)>1:
            for slurp in slurps[1:]:
                f.write('\n' + str(slurp))
        
    with open(file_name) as f:    
        soup = BeautifulSoup(f.read())
#     pd.to_csv(slurps, header=None, index=False)

    return soup

In [22]:
soup = get_soup(url='https://github.com/search?o=desc&p=1&q=advent+of+code&s=stars&type=Repositories', soup_slurper='.repo-list-item')

In [7]:
def all_repositories():
    page = str(range(1,21))
    repo_list = []
    for p in page:
        repo_list.extend(get_links_from_page('https://github.com/search?o=desc&p='+p+'&q=advent+of+code&s=stars&type=Repositories'))
        return repo_list

In [8]:
def get_links_from_page(url):
    soup = get_soup(
    url=url,
    cache=True,
    soup_slurper='.repo-list-item a',
    file_name='soupfiles/gitsearch.txt'
)

In [22]:
df = pd.read_csv('json_test.csv')
df.readme_contents[6]

'# Advent of Code Elixir Starter\n\nA batteries included starter pack for participating in [Advent of Code](https://www.adventofcode.com) using Elixir!\n\n## Usage\n\nThere are 25 modules, 25 tests, and 50 mix tasks. \n\n1. Fill in the tests with the example solutions.\n1. Write your implementation.\n1. Fill in the final problem inputs into the mix task and run `mix d01.p1`!\n    - Benchmark your solution by passing the `-b` flag, `mix d01.p1 -b`\n\n```elixir\ndefmodule AdventOfCode.Day01 do\n  def part1(args) do\n  end\n\n  def part2(args) do\n  end\nend\n```\n\n```elixir\ndefmodule AdventOfCode.Day01Test do\n  use ExUnit.Case\n\n  import AdventOfCode.Day01\n\n  @tag :skip # Make sure to remove to run your test.\n  test "part1" do\n    input = nil \n    result = part1(input)\n\n    assert result\n  end\n\n  @tag :skip # Make sure to remove to run your test.\n  test "part2" do\n    input = nil \n    result = part2(input)\n\n    assert result\n  end\nend\n```\n\n```elixir\ndefmodule Mix

In [51]:
# create custom string to test conditions and then lower case the string
string = 'C++ , C+ , C#' + df.readme_contents[6]
string = string.lower()

# replace new lines with an empty space
string = re.sub(r'\n',' ',string)

# replace c++ with c plus plus
string = re.sub(r'c\+\+','cplusplus',string)

# replace c+ with c plus
string = re.sub(r'c\+','cplus ',string)

# replace c# with c sharp
string = re.sub(r'c#','csharp ',string)

# replace extra spaces with only one space
string = re.sub(r'\s+',' ',string)

# remove special characters with regex
string = re.sub(r'[^a-z0-9\s]','', string)
string


'c plusplus  c plus  c sharp  advent of code elixir starter a batteries included starter pack for participating in advent of codehttpswwwadventofcodecom using elixir  usage there are 25 modules 25 tests and 50 mix tasks 1 fill in the tests with the example solutions 1 write your implementation 1 fill in the final problem inputs into the mix task and run mix d01p1  benchmark your solution by passing the b flag mix d01p1 b elixir defmodule adventofcodeday01 do def part1args do end def part2args do end end  elixir defmodule adventofcodeday01test do use exunitcase import adventofcodeday01 tag skip  make sure to remove to run your test test part1 do input  nil result  part1input assert result end tag skip  make sure to remove to run your test test part2 do input  nil result  part2input assert result end end  elixir defmodule mixtasksd01p1 do use mixtask import adventofcodeday01 shortdoc day 01 part 1 def runargs do input  nil if enummemberargs b do bencheerunpart1 fn  input  part1 end else 

In [50]:
string

'c plus plus  c plus  c sharp  advent of code elixir starter a batteries included starter pack for participating in advent of codehttpswwwadventofcodecom using elixir  usage there are 25 modules 25 tests and 50 mix tasks 1 fill in the tests with the example solutions 1 write your implementation 1 fill in the final problem inputs into the mix task and run mix d01p1  benchmark your solution by passing the b flag mix d01p1 b elixir defmodule adventofcodeday01 do def part1args do end def part2args do end end  elixir defmodule adventofcodeday01test do use exunitcase import adventofcodeday01 tag skip  make sure to remove to run your test test part1 do input  nil result  part1input assert result end tag skip  make sure to remove to run your test test part2 do input  nil result  part2input assert result end end  elixir defmodule mixtasksd01p1 do use mixtask import adventofcodeday01 shortdoc day 01 part 1 def runargs do input  nil if enummemberargs b do bencheerunpart1 fn  input  part1 end else